# Reactant demo

In [1]:
using Tenet
using EinExprs
using Reactant
using AbstractTrees
using Random
using LinearAlgebra
using Enzyme
using Chairmarks
using BenchmarkTools
using Adapt
BLAS.set_num_threads(1)

In [2]:
Random.seed!(0)
tn = rand(TensorNetwork, 15, 3; dim=(16,16))

TensorNetwork (#tensors=15, #inds=22)

In [3]:
path = einexpr(tn; optimizer=Exhaustive())

SizedEinExpr{Symbol}(EinExpr{Symbol}(Symbol[], EinExpr{Symbol}[EinExpr{Symbol}([:R], EinExpr{Symbol}[EinExpr{Symbol}([:K], EinExpr{Symbol}[]), EinExpr{Symbol}([:R, :K], EinExpr{Symbol}[])]), EinExpr{Symbol}([:R], EinExpr{Symbol}[EinExpr{Symbol}([:G, :N, :R], EinExpr{Symbol}[]), EinExpr{Symbol}([:G, :N], EinExpr{Symbol}[EinExpr{Symbol}([:H, :Q, :N], EinExpr{Symbol}[]), EinExpr{Symbol}([:H, :Q, :G], EinExpr{Symbol}[EinExpr{Symbol}([:C, :F, :Q], EinExpr{Symbol}[]), EinExpr{Symbol}([:H, :C, :F, :G], EinExpr{Symbol}[EinExpr{Symbol}([:D, :H, :O, :C], EinExpr{Symbol}[]), EinExpr{Symbol}([:D, :O, :F, :G], EinExpr{Symbol}[EinExpr{Symbol}([:B, :D, :F, :J], EinExpr{Symbol}[EinExpr{Symbol}([:B, :D, :F, :V], EinExpr{Symbol}[]), EinExpr{Symbol}([:V, :J], EinExpr{Symbol}[EinExpr{Symbol}([:P, :J], EinExpr{Symbol}[]), EinExpr{Symbol}([:V, :P], EinExpr{Symbol}[EinExpr{Symbol}([:E], EinExpr{Symbol}[]), EinExpr{Symbol}([:E, :V, :P], EinExpr{Symbol}[])])])]), EinExpr{Symbol}([:B, :O, :G, :J], EinExpr{Symbo

In [4]:
@benchmark contract(tn; path)

BenchmarkTools.Trial: 411 samples with 1 evaluation.
 Range (min … max):  10.154 ms … 228.945 ms  ┊ GC (min … max): 0.00% … 95.24%
 Time  (median):     10.357 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.134 ms ±  10.944 ms  ┊ GC (mean ± σ):  5.96% ±  5.90%

  █▇▃           ▁   ▁   ▂                                       
  ███▇▇▄▄▄▁▄▁▁▁▇██▅▁██▇▇██▅▄▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇██▇▁▅▁▁▄▁▁▁▄▅▆▇▇█ ▇
  10.2 ms       Histogram: log(frequency) by time      17.5 ms <

 Memory estimate: 9.41 MiB, allocs estimate: 3303.

In [5]:
tn′ = adapt(Reactant.ConcreteRArray, tn)
g = Reactant.compile(Tuple(tensors(tn′))) do ts...
    _tn = TensorNetwork(ts)
    contract(_tn; path)
end

Reactant.Thunk{Val{(((:result, 1),),)}, Val{(((:args, 8, 1), (:resargs, 8, 1)), ((:args, 11, 1), (:resargs, 11, 1)), ((:args, 2, 1), (:resargs, 2, 1)), ((:args, 5, 1), (:resargs, 5, 1)), ((:args, 13, 1), (:resargs, 13, 1)), ((:args, 15, 1), (:resargs, 15, 1)), ((:args, 4, 1), (:resargs, 4, 1)), ((:args, 6, 1), (:resargs, 6, 1)), ((:args, 1, 1), (:resargs, 1, 1)), ((:args, 14, 1), (:resargs, 14, 1)), ((:args, 7, 1), (:resargs, 7, 1)), ((:args, 12, 1), (:resargs, 12, 1)), ((:args, 3, 1), (:resargs, 3, 1)), ((:args, 9, 1), (:resargs, 9, 1)), ((:args, 10, 1), (:resargs, 10, 1)))}, Val{((((:args, 12, 1), (:resargs, 12, 1)), 11), (((:args, 10, 1), (:resargs, 10, 1)), 14), (((:args, 4, 1), (:resargs, 4, 1)), 6), (((:args, 9, 1), (:resargs, 9, 1)), 13), (((:args, 15, 1), (:resargs, 15, 1)), 5), (((:args, 14, 1), (:resargs, 14, 1)), 9), (((:args, 8, 1), (:resargs, 8, 1)), 0), (((:args, 1, 1), (:resargs, 1, 1)), 8), (((:args, 13, 1), (:resargs, 13, 1)), 4), (((:args, 5, 1), (:resargs, 5, 1)), 3)

In [6]:
@benchmark g(tensors(tn′)...)

BenchmarkTools.Trial: 564 samples with 1 evaluation.
 Range (min … max):  8.199 ms …  19.417 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.747 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.861 ms ± 776.079 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁▄▂▆██▆█▅▃▄▂ ▂                                           
  ▃▃▆▆██████████████▇▇▆▆▅▄▃▃▄▂▁▃▁▂▁▂▁▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▄
  8.2 ms          Histogram: frequency by time          11 ms <

 Memory estimate: 496 bytes, allocs estimate: 7.

In [7]:
function f(x...)
    _tn = TensorNetwork(x)
    contract(_tn; path)
end

∇g = Reactant.compile(Tuple(tensors(tn′))) do x...
    dx = Enzyme.make_zero.(x)
    Enzyme.autodiff(Reverse, f, Active, Duplicated.(x,dx)...)
    return dx
end

Reactant.Thunk{Val{(((:result, 4, 1),), ((:result, 14, 1),), ((:result, 9, 1),), ((:result, 7, 1),), ((:result, 6, 1),), ((:result, 2, 1),), ((:result, 10, 1),), ((:result, 15, 1),), ((:result, 3, 1),), ((:result, 5, 1),), ((:result, 12, 1),), ((:result, 8, 1),), ((:result, 1, 1),), ((:result, 13, 1),), ((:result, 11, 1),))}, Val{(((:args, 8, 1), (:resargs, 8, 1)), ((:args, 11, 1), (:resargs, 11, 1)), ((:args, 2, 1), (:resargs, 2, 1)), ((:args, 5, 1), (:resargs, 5, 1)), ((:args, 13, 1), (:resargs, 13, 1)), ((:args, 15, 1), (:resargs, 15, 1)), ((:args, 4, 1), (:resargs, 4, 1)), ((:args, 6, 1), (:resargs, 6, 1)), ((:args, 1, 1), (:resargs, 1, 1)), ((:args, 14, 1), (:resargs, 14, 1)), ((:args, 7, 1), (:resargs, 7, 1)), ((:args, 12, 1), (:resargs, 12, 1)), ((:args, 3, 1), (:resargs, 3, 1)), ((:args, 9, 1), (:resargs, 9, 1)), ((:args, 10, 1), (:resargs, 10, 1)))}, Val{((((:args, 9, 1), (:resargs, 9, 1)), 13), (((:args, 7, 1), (:resargs, 7, 1)), 10), (((:args, 8, 1), (:resargs, 8, 1)), 0), (

In [8]:
@benchmark ∇g(tensors(tn′)...)

BenchmarkTools.Trial: 186 samples with 1 evaluation.
 Range (min … max):  20.952 ms … 46.400 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     25.675 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   26.854 ms ±  4.545 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▁▃█▁▂ ▂▅▇▂▅                                               
  ▃▃▆███████████▇█▄▇▆▆▆█▆▇▃▃▄▃▃▃▄▃▃▁▁▁▄▃▃▃▁▁▃▁▃▁▁▃▁▃▃▁▁▁▁▁▁▁▃ ▃
  21 ms           Histogram: frequency by time        44.4 ms <

 Memory estimate: 2.64 KiB, allocs estimate: 63.

In [9]:
re∇g = Reactant.compile(Tuple(tensors(tn′))) do x...
    dx = Enzyme.make_zero.(x)
    primal = Enzyme.autodiff(ReverseWithPrimal, f, Active, Duplicated.(x,dx)...)
    return (primal, dx)
end

Reactant.Thunk{Val{(((:result, 2, 14, 1),), ((:result, 1, 2, 1),), ((:result, 2, 15, 1),), ((:result, 2, 6, 1),), ((:result, 2, 10, 1),), ((:result, 2, 9, 1),), ((:result, 2, 13, 1),), ((:result, 2, 5, 1),), ((:result, 2, 8, 1),), ((:result, 2, 12, 1),), ((:result, 2, 1, 1),), ((:result, 2, 3, 1),), ((:result, 2, 7, 1),), ((:result, 2, 4, 1),), ((:result, 2, 11, 1),), ((:result, 2, 2, 1),))}, Val{(((:args, 8, 1), (:resargs, 8, 1)), ((:args, 11, 1), (:resargs, 11, 1)), ((:args, 2, 1), (:resargs, 2, 1)), ((:args, 5, 1), (:resargs, 5, 1)), ((:args, 13, 1), (:resargs, 13, 1)), ((:args, 15, 1), (:resargs, 15, 1)), ((:args, 4, 1), (:resargs, 4, 1)), ((:args, 6, 1), (:resargs, 6, 1)), ((:args, 1, 1), (:resargs, 1, 1)), ((:args, 14, 1), (:resargs, 14, 1)), ((:args, 7, 1), (:resargs, 7, 1)), ((:args, 12, 1), (:resargs, 12, 1)), ((:args, 3, 1), (:resargs, 3, 1)), ((:args, 9, 1), (:resargs, 9, 1)), ((:args, 10, 1), (:resargs, 10, 1)))}, Val{((((:args, 8, 1), (:resargs, 8, 1)), 0), (((:args, 2, 1)

In [10]:
@benchmark re∇g(tensors(tn′)...)

BenchmarkTools.Trial: 184 samples with 1 evaluation.
 Range (min … max):  25.174 ms … 42.356 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     26.660 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   27.289 ms ±  2.104 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▄█▁                                                     
  ▄▅▅▆▆███▇▇▅▆▇▃▄▅▃▅▃▂▂▂▁▁▂▂▁▁▁▁▂▁▁▂▁▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂ ▂
  25.2 ms         Histogram: frequency by time        37.3 ms <

 Memory estimate: 2.77 KiB, allocs estimate: 67.